# Procedural Modeling of Buildings



In [1]:
import os
import sys
import time
import pickle

In [2]:
# add udtools library to path
home_dir = os.environ.get('HOME')
udtools_module_path = os.path.join(home_dir, 'git', 'ud-tools-core', 'udtools')

if udtools_module_path not in sys.path:
  sys.path.append(udtools_module_path)

In [3]:
import ipywidgets as widgets

# OCC/pythonocc-core imports
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

# udtools imports
from archtools.geometry.boolean import diff
from archtools.geometry.common import get_faces
from archtools.material.color import random_color
from nyczoning.zoninglot import ZoningLot
from nyczoning.geometry import build_yard_cutter
from nyclanduse.site import Site
from nyclanduse.scenario import Scenario
from nyclanduse.study import Study

In [4]:
# load a zoning lot, this will have an underlying boundary and envelope
with open('./assets/20210928_zl.pickle', 'rb') as f:
    zl = pickle.load(f)

In [5]:
# here's what we've got:
renderer2 = JupyterRenderer()
renderer2.DisplayShape(zl.geom, render_edges=True, topo_level="default", shape_color="#abdda4", update=True)

In [6]:
# render the envelope as individual faces by color:
faces = get_faces(zl.envelope)

for f in faces:
    renderer2.DisplayShape(f, shape_color=random_color())
    time.sleep(0.25)

## Classified Faces from Envelope

Create a `BuildingEnvelope` class to get classified faces and edges from the zoning envelope. Some issues/questions that have come up through the development of this:

- How does Orientation and Closed-ness work in OCC? It was initially unclear if the `TopoDS_Compound` object returned by the zoning lot generator is oriented correctly. The shape itself returns 0 when Closed() is called, which at first seems to mean "not closed" but apparently actually returns the result of BRepCheck_NoError so it really means the Shape is free of errors (see [this](https://dev.opencascade.org/content/shell-closedness)).
- Using ShapeFix, using the underlying Shell or Solid in place of the original Compound, and other methods for ensuring correct orientation of faces has no effect on the result.
- The main issue turned out to be that in OpenCASCADE the orientations of faces and their underlying surfaces can be different, since the face orientation os a product of the direction of the wires that define its boundary, and that orientation depends on connectivity with the shape as a whole rather than underlying geometry. For discussion see [this blog post series](https://opencascade.blogspot.com/2009/02/continued.html) The `face_normal` function in `archtools` was edited to take this fact into account and now works as expected.

The `classify_faces` method of a `BuildingEnvelope` iterates over the faces in the provided Shape and returns a dictionary of tagged faces.

In [7]:
from archtools.building.envelope import BuildingEnvelope

In [8]:
envelope = BuildingEnvelope(zl.envelope)
envelope.lot = zl
faces_classed = envelope.classify_faces()
faces_classed

{'belly': [[<class 'TopoDS_Face'>, False]],
 'facade': [[<class 'TopoDS_Face'>, True],
  [<class 'TopoDS_Face'>, True],
  [<class 'TopoDS_Face'>, True],
  [<class 'TopoDS_Face'>, True],
  [<class 'TopoDS_Face'>, False],
  [<class 'TopoDS_Face'>, False]],
 'roof': [[<class 'TopoDS_Face'>, False],
  [<class 'TopoDS_Face'>, False],
  [<class 'TopoDS_Face'>, False]]}

In [9]:
renderer3 = JupyterRenderer()
renderer3.DisplayShape(zl.geom, render_edges=True, topo_level="default", shape_color="#abdda4", update=True)

In [10]:
for f in faces_classed['facade']:
    if f[1]:
        renderer3.DisplayShape(f[0], shape_color='#e6d3a5')
    else:
        renderer3.DisplayShape(f[0], shape_color='#dbb69e')
    time.sleep(0.25)

In [11]:
for f in faces_classed['roof']:
    renderer3.DisplayShape(f[0], shape_color='#5e5b5a')
    time.sleep(0.25)

In [12]:
for f in faces_classed['belly']:
    renderer3.DisplayShape(f[0], shape_color='#ff0000')
    time.sleep(0.25)

In [13]:
###

In [14]:
from archtools.geometry.face import face_centroid
from archtools.geometry.wire import to_curve_from_vertices
from archtools.geometry.curve import pt_on_curve

In [15]:
bounds_curve = to_curve_from_vertices(zl.geom)
face_pt = face_centroid(faces_classed['roof'][0][0])

In [16]:
face_pt

<class 'gp_Pnt'>

In [17]:
pt_on_curve(face_pt, bounds_curve, 8)

True

## Get select edges/wires for parapets, fences, cornices and stoops

With conditions and trimmed by intersections, would allow to get basis curves for parapets, fences etc. See "get faces belonging to an edge" on OpenCASCADE forums.

In [18]:
# roof_edges = get_edges(roof)
# for e in roof_edges:
#   print(e)

In [19]:
from OCC.Core.TopExp import topexp_MapShapesAndAncestors
from OCC.Core.TopTools import (
  TopTools_IndexedDataMapOfShapeListOfShape,
)
from OCC.Core.TopOpeBRepBuild import TopOpeBRepBuild_Tools
from OCC.Core.TopAbs import TopAbs_EDGE, TopAbs_FACE

edge_face_map = TopTools_IndexedDataMapOfShapeListOfShape()
topexp_MapShapesAndAncestors(roof, TopAbs_EDGE, TopAbs_FACE, edge_face_map)
edge_face_map

#     TopExp
#       MapShapesAndAncestors(shell, TopAbs_EDGE,
#             TopAbs_FACE, edgeFaceMap);

NameError: name 'roof' is not defined

In [ ]:
edge_face_map.Size() # size is 4
# list_at_1 = edge_face_map.FindFromIndex(1) # indexes are 1-indexed
# list_at_1.Size() this is 1

# list_at_ = edge_face_map.FindKey

## Generate facade detail on faces as a shell and replace original




## Checklist

- [x] ensure Solid orientation, will fix face orientation and directional classification? *turns out this was actually an issue with interpretation of the normals, rather than a problem with the original Shape, thankfully. fixed.*
- [ ] get_edge methods for parapets, fences
- [ ] draw facades
- [ ] revisit "Facade Markup Language" idea, consider Tidalcycles and other livecoding languages as precedent
- [ ] next notebook: procedural streets?